In [2]:
! pip install langchain_community tiktoken langchain-groq langchainhub chromadb langchain langchain gpt4all

  Using cached langchain_community-0.0.37-py3-none-any.whl (2.0 MB)
  Using cached tiktoken-0.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)
  Using cached langchain_groq-0.1.3-py3-none-any.whl (11 kB)
  Using cached langchainhub-0.1.15-py3-none-any.whl (4.6 kB)
  Using cached chromadb-0.5.0-py3-none-any.whl (526 kB)
  Using cached langchain-0.1.17-py3-none-any.whl (867 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 23.5 MB/s eta 0:00:00
  Using cached dataclasses_json-0.6.5-py3-none-any.whl (28 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.

In [4]:
import os
os.environ['GROQ_API_KEY'] = "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"

In [9]:
from langchain_community.embeddings import GPT4AllEmbeddings
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_groq import ChatGroq



#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GPT4AllEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is AutoGPT?")

"AutoGPT is an autonomous agent with a natural language interface, using a large language model as its main controller. It has drawn attention for its potential in creating autonomous agents, but it has reliability issues due to the natural language interface. AutoGPT's goals include making decisions independently and pursuing simple strategies with no legal complications."

In [10]:
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [11]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")


8

In [12]:
from langchain_community.embeddings import GPT4AllEmbeddings
embd = GPT4AllEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

384

In [13]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.5783774219799266


In [14]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [15]:

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [16]:
# Index
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GPT4AllEmbeddings())

retriever = vectorstore.as_retriever()

In [17]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GPT4AllEmbeddings())


retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [18]:
docs = retriever.get_relevant_documents("What is Task Decomposition?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [19]:
len(docs)

1

In [20]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [21]:
llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0)

In [22]:

# Chain
chain = prompt | llm

In [23]:
# Run
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

AIMessage(content='Task Decomposition, as described in the provided document, is a process used in autonomous agent systems to break down a complicated task into smaller and more manageable steps. This can be done using a large language model (LLM) with simple prompting, such as asking for "Steps for XYZ" or "What are the subgoals for achieving XYZ?" It can also be done using task-specific instructions or with human inputs. Task Decomposition is a key component of the planning process in an autonomous agent system.', response_metadata={'token_usage': {'completion_time': 0.202, 'completion_tokens': 113, 'prompt_time': 0.343, 'prompt_tokens': 385, 'queue_time': None, 'total_time': 0.545, 'total_tokens': 498}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-fd81cc07-3daa-478e-8a89-092d2953b9e9-0')

In [24]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [25]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [26]:

prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition, as described in the provided document, is a process used in autonomous agent systems to break down a complicated task into smaller and more manageable steps. This can be done using a large language model (LLM) with simple prompting, task-specific instructions, or human inputs. The technique of "Chain of Thought" (CoT) involves instructing the model to "think step by step" to decompose hard tasks into smaller steps, providing insights into the model\'s thinking process. The "Tree of Thoughts" approach further extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thoughts.'